In [2]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.stats import norm

In [3]:
df_read = pd.read_csv('C:/Users/Erlend/Google Drive/NTNU/5. Klasse/Master/Git-folder/deep-learning-for-option-pricing/data/results/combined_predictions2.csv')

# Filter Quote_date only in 2018
#df = df[df['Quote_date'].str.contains('2018')]
'''df = df[df['Quote_date'] >= '2018-10-01']
df = df[df['Quote_date'] < '2018-10-31']'''

df_read['Expiry_date'] = pd.to_datetime(df_read['Quote_date']) + pd.to_timedelta(df_read['TTM']*365, unit='D')
df_read['Expiry_date'] = pd.to_datetime(df_read['Expiry_date'])
df_read['Quote_date'] = pd.to_datetime(df_read['Quote_date'])

# Adding option ID
df_read["Option_ID"] = df_read["Expiry_date"].astype(str) + "-" + df_read["Strike"].astype(str)


print(df_read['Quote_date'].min())
print(df_read['Quote_date'].max())

print(df_read.head(3))
df_read.info()

2015-01-02 00:00:00
2023-03-31 00:00:00
   Unnamed: 0_x Quote_date    Price  Underlying_last_x  Strike       TTM  \
0       4604456 2017-11-24  178.250            2602.42  2425.0  0.076712   
1       4604548 2017-11-24  451.595            2602.42  2150.0  0.087671   
2       4605032 2017-11-24  432.895            2602.42  2170.0  0.115068   

        R_x  rolling_volatility  Volatility_x     Rolling  ...  \
0  0.011400            0.052645      0.052645  179.539789  ...   
1  0.011454            0.052645      0.052645  454.577919  ...   
2  0.011730            0.052645      0.052645  435.346946  ...   

   Underlying_last_y.1     R_y.1    Month      Heston  Unnamed: 0   LSTM-MLP  \
0              2602.42  0.011400  2017-11  180.125992     4022502  181.22504   
1              2602.42  0.011454  2017-11  453.536094     4022594  452.85504   
2              2602.42  0.011730  2017-11  434.552391     4023070  433.30730   

   Underlying_last         R  Expiry_date          Option_ID  
0     

In [4]:
df_read = df_read.sort_values(by=['Quote_date', 'Strike', 'TTM'], ascending=True)

df_sp = df_read[['Quote_date', 'Underlying_last']].groupby('Quote_date').mean().reset_index()

# Rename underlying last to Sp_expiry
df_sp.rename(columns={'Underlying_last': 'SP_expiry', 'Quote_date': 'Expiry_date'}, inplace=True)

# Merge df_sp back to df on expiry date
df_read = df_read.merge(df_sp, how='left', on='Expiry_date')

df_read['YearMonth'] = df_read['Quote_date'].dt.to_period('M')

print(df_read.head(3))
print(df_read.info())


   Unnamed: 0_x Quote_date     Price  Underlying_last_x  Strike       TTM  \
0       1809055 2015-01-02  1007.250            2058.69  1050.0  0.019178   
1       1809243 2015-01-02  1007.295            2058.69  1050.0  0.035616   
2       1809438 2015-01-02  1006.650            2058.69  1050.0  0.057534   

      R_x  rolling_volatility  Volatility_x      Rolling  ...    Month  \
0  0.0002            0.130943      0.130943  1008.694027  ...  2015-01   
1  0.0002            0.130943      0.130943  1008.697479  ...  2015-01   
2  0.0002            0.130943      0.130943  1008.702082  ...  2015-01   

        Heston  Unnamed: 0   LSTM-MLP  Underlying_last       R  Expiry_date  \
0  1007.997058     1518156  1010.0069          2058.69  0.0002   2015-01-09   
1  1007.525045     1518320  1009.4740          2058.69  0.0002   2015-01-15   
2  1006.973740     1518511  1008.7636          2058.69  0.0002   2015-01-23   

           Option_ID  SP_expiry  YearMonth  
0  2015-01-09-1050.0    2044.93 

In [7]:
def black_scholes_delta(S, K, T, r, sigma = 0.3):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

df_read['Delta'] = black_scholes_delta(df_read['Underlying_last'].astype(float), df_read['Strike'].astype(float), df_read['TTM'].astype(float), df_read['R'].astype(float))
df_read['Option_exercise'] = (df_read['SP_expiry'] - df_read['Strike']).clip(lower=0)

df_read['Hedge_cost'] = df_read['Delta'] * df_read['Underlying_last']
df_read['Total_cost'] = df_read['Price'] + df_read['Hedge_cost']

print(df_read.head())
df_read.info()

   Unnamed: 0_x Quote_date     Price  Underlying_last_x  Strike       TTM  \
0       1809055 2015-01-02  1007.250            2058.69  1050.0  0.019178   
1       1809243 2015-01-02  1007.295            2058.69  1050.0  0.035616   
2       1809438 2015-01-02  1006.650            2058.69  1050.0  0.057534   
3       1809599 2015-01-02  1006.755            2058.69  1050.0  0.076712   
4       1809754 2015-01-02  1005.205            2058.69  1050.0  0.095890   

        R_x  rolling_volatility  Volatility_x      Rolling  ...  \
0  0.000200            0.130943      0.130943  1008.694027  ...   
1  0.000200            0.130943      0.130943  1008.697479  ...   
2  0.000200            0.130943      0.130943  1008.702082  ...   
3  0.000200            0.130943      0.130943  1008.706109  ...   
4  0.000139            0.130943      0.130943  1008.704004  ...   

   Underlying_last         R  Expiry_date          Option_ID  SP_expiry  \
0          2058.69  0.000200   2015-01-09  2015-01-09-1050.

In [35]:
transaction_cost_options = 0.002
transaction_cost_stock = 0.002
rf = 0.02

def backtest(df):
    # Lists to store returns
    monthly_returns = {
        'long': np.array([]),
        'short': np.array([]),
        'long_hedge': np.array([]),
        'short_hedge': np.array([]),
        'long_portfolio': np.array([]),
        'short_portfolio': np.array([]),
        'hedge_agg': np.array([]),
        'hedge_agg_portfolio': np.array([]),
    }

    long_expired_share = np.array([])
    short_expired_share = np.array([])

    rf = 0.02

    # Group by month
    grouped = df.groupby('YearMonth')

    for name, group in grouped:
        first_day = group['Quote_date'].iloc[0]
        df_start = group[group['Quote_date'] == first_day]
        df_start = df_start[df_start['Price'] >= 1]
        #df_start = df_start[df_start['TTM'] >= 5/365]

        last_trading_day = group['Quote_date'].iloc[-1]
        #last_trading_day = group['Quote_date'].unique()[5]
        df_final = group[group['Quote_date'] == last_trading_day][['Price', 'Option_ID']]
        df_final.rename(columns={'Price': 'End_price'}, inplace=True)    
        df_start = df_start.merge(df_final, how='left', on='Option_ID')
        df_start = df_start[~((df_start['Expiry_date'] > last_trading_day) & (df_start['End_price'].isnull()))]

        # Select the top 10% underpriced and overpriced options
        underpriced = df_start.nlargest(int(0.1*len(df_start)), 'Relative_pricing_error')
        overpriced = df_start.nsmallest(int(0.1*len(df_start)), 'Relative_pricing_error')

        investment_amount = 1000000
        underpriced['Quantity']= investment_amount / len(underpriced) / underpriced['Price']
        overpriced['Quantity'] = investment_amount / len(overpriced) / overpriced['Price']

        '''underpriced = df_start[df_start['Absolute_pricing_error'] >= 1]
        overpriced = df_start[df_start['Absolute_pricing_error'] <= -1]'''
        
        underpriced_expired = underpriced[underpriced['Expiry_date'] <= last_trading_day]
        overpriced_expired = overpriced[overpriced['Expiry_date'] <= last_trading_day]

        underpriced_alive = underpriced[underpriced['Expiry_date'] > last_trading_day]
        overpriced_alive = overpriced[overpriced['Expiry_date'] > last_trading_day]

        long_expired_share = np.append(long_expired_share, len(underpriced_expired)/len(underpriced))
        short_expired_share = np.append(short_expired_share, len(overpriced_expired)/len(overpriced))

        #Calculate abs returns
        underpriced_expired.loc[:, 'Option_return_abs'] = underpriced_expired['Option_exercise'] - underpriced_expired['Price'] - transaction_cost_options*underpriced_expired['Price']
        overpriced_expired.loc[:, 'Option_return_abs'] = -(overpriced_expired['Option_exercise'] - overpriced_expired['Price']) - transaction_cost_options*overpriced_expired['Price']

        underpriced_alive.loc[:, 'Option_return_abs'] = (underpriced_alive['End_price'] - underpriced_alive['Price']) - transaction_cost_options*(underpriced_alive['Price'] + underpriced_alive['End_price'])
        overpriced_alive.loc[:, 'Option_return_abs'] = - (overpriced_alive['End_price'] - overpriced_alive['Price']) - transaction_cost_options*(overpriced_alive['Price'] + overpriced_alive['End_price'])

        underpriced_expired.loc[:, 'Hedge_return_abs'] = -underpriced_expired['Delta'] * (underpriced_expired['SP_expiry'] - underpriced_expired['Underlying_last']) - transaction_cost_stock*underpriced_expired['Delta']*(underpriced_expired['SP_expiry'] + underpriced_expired['Underlying_last'])
        overpriced_expired.loc[:, 'Hedge_return_abs'] = overpriced_expired['Delta'] * (overpriced_expired['SP_expiry'] - overpriced_expired['Underlying_last']) - transaction_cost_stock*overpriced_expired['Delta']*(overpriced_expired['SP_expiry'] + overpriced_expired['Underlying_last'])

        sp500_final = group[group['Quote_date'] == last_trading_day]['Underlying_last'].iloc[-1]
        underpriced_alive.loc[:, 'Hedge_return_abs'] = - underpriced_alive['Delta'] * (sp500_final - underpriced_alive['Underlying_last']) - transaction_cost_stock*underpriced_alive['Delta']*(sp500_final + underpriced_alive['Underlying_last'])
        overpriced_alive.loc[:, 'Hedge_return_abs'] = overpriced_alive['Delta'] * (sp500_final - overpriced_alive['Underlying_last']) - transaction_cost_stock*overpriced_alive['Delta']*(sp500_final + overpriced_alive['Underlying_last'])

        # Calculate pct returns
        underpriced_expired.loc[:, 'Option_return_pct'] = underpriced_expired['Option_return_abs'] / underpriced_expired['Price']
        overpriced_expired.loc[:, 'Option_return_pct'] = overpriced_expired['Option_return_abs'] / overpriced_expired['Price']

        underpriced_alive.loc[:, 'Option_return_pct'] = underpriced_alive['Option_return_abs'] / underpriced_alive['Price']
        overpriced_alive.loc[:, 'Option_return_pct'] = overpriced_alive['Option_return_abs'] / overpriced_alive['Price']

        underpriced_expired.loc[:, 'Hedge_return_pct'] = underpriced_expired['Hedge_return_abs'] / (underpriced_expired['Delta']*underpriced_expired['Underlying_last'])
        overpriced_expired.loc[:, 'Hedge_return_pct'] = overpriced_expired['Hedge_return_abs'] / (overpriced_expired['Delta']*overpriced_expired['Underlying_last'])

        underpriced_alive.loc[:, 'Hedge_return_pct'] = underpriced_alive['Hedge_return_abs'] / (underpriced_alive['Delta']*underpriced_alive['Underlying_last'])
        overpriced_alive.loc[:, 'Hedge_return_pct'] = overpriced_alive['Hedge_return_abs'] / (overpriced_alive['Delta']*overpriced_alive['Underlying_last'])

        underpriced_expired.loc[:, 'Portfolio_return_pct'] = (underpriced_expired['Option_return_abs'] + underpriced_expired['Hedge_return_abs']) / (underpriced_expired['Price'] + underpriced_expired['Hedge_cost'])
        overpriced_expired.loc[:, 'Portfolio_return_pct'] = (overpriced_expired['Option_return_abs'] + overpriced_expired['Hedge_return_abs']) / (overpriced_expired['Price'] + overpriced_expired['Hedge_cost'])

        underpriced_alive.loc[:, 'Portfolio_return_pct'] = (underpriced_alive['Option_return_abs'] + underpriced_alive['Hedge_return_abs']) / (underpriced_alive['Price'] + underpriced_alive['Hedge_cost'])
        overpriced_alive.loc[:, 'Portfolio_return_pct'] = (overpriced_alive['Option_return_abs'] + overpriced_alive['Hedge_return_abs']) / (overpriced_alive['Price'] + overpriced_alive['Hedge_cost'])

        # Delta heding portfolio
        sp500_first = group['Underlying_last'].iloc[0]

        delta = (overpriced['Delta']*overpriced['Quantity']).sum() - (underpriced['Delta']*underpriced['Quantity']).sum()

        hedge_agg_abs = delta * (sp500_final - sp500_first) - transaction_cost_stock*abs(delta)*(sp500_first + sp500_final)
        hedge_cost = abs(delta)*sp500_first
        hedge_agg_pct = hedge_agg_abs / hedge_cost

        return_long = (underpriced_expired['Option_return_pct'].sum() + underpriced_alive['Option_return_pct'].sum()) / len(underpriced)
        return_short = (overpriced_expired['Option_return_pct'].sum() + overpriced_alive['Option_return_pct'].sum()) / len(overpriced)

        tot_invest = 2*investment_amount + hedge_cost

        portfolio_agg_hedged_pct = return_long * investment_amount / tot_invest + return_short * investment_amount / tot_invest + hedge_agg_pct * hedge_cost / tot_invest

        monthly_returns['long'] = np.append(monthly_returns['long'], return_long)
        monthly_returns['short'] = np.append(monthly_returns['short'], return_short)
        monthly_returns['long_hedge'] = np.append(monthly_returns['long_hedge'], (underpriced_expired['Hedge_return_pct'].sum() + underpriced_alive['Hedge_return_pct'].sum()) / len(underpriced))
        monthly_returns['short_hedge'] = np.append(monthly_returns['short_hedge'], (overpriced_expired['Hedge_return_pct'].sum() + overpriced_alive['Hedge_return_pct'].sum()) / len(overpriced))
        monthly_returns['long_portfolio'] = np.append(monthly_returns['long_portfolio'], (underpriced_expired['Portfolio_return_pct'].sum() + underpriced_alive['Portfolio_return_pct'].sum()) / len(underpriced))
        monthly_returns['short_portfolio'] = np.append(monthly_returns['short_portfolio'], (overpriced_expired['Portfolio_return_pct'].sum() + overpriced_alive['Portfolio_return_pct'].sum()) / len(overpriced))
        monthly_returns['hedge_agg'] = np.append(monthly_returns['hedge_agg'], hedge_agg_pct)
        monthly_returns['hedge_agg_portfolio'] = np.append(monthly_returns['hedge_agg_portfolio'], portfolio_agg_hedged_pct)

    # Add combined returns
    monthly_returns['combined'] = 0.5*monthly_returns['long'] + 0.5*monthly_returns['short']
    monthly_returns['combined_hedge'] = 0.5*monthly_returns['long_hedge'] + 0.5*monthly_returns['short_hedge']
    monthly_returns['combined_hedged_portfolio'] = 0.5*monthly_returns['long_portfolio'] + 0.5*monthly_returns['short_portfolio']

    monthly_returns = {
        'long': np.array([monthly_returns['long']])*0.2,
        'short': np.array([monthly_returns['short']])*0.2,
        'combined': np.array([monthly_returns['combined']])*0.2,
    }


    # Print values
    '''for values in monthly_returns.items():
        print(values)'''

    # Print average monthly returns
    for key, values in monthly_returns.items():
        print(f'Average monthly return {key} {np.mean(values)*100:.2f} %')

    '''# Print annualized returns
    for key, values in monthly_returns.items():
        annualized = (1 + values) ** 12 - 1
        print(f'Annualized return {key} {np.mean(annualized)*100:.2f}')'''

    '''# Print Sharpe ratios
    for key, values in monthly_returns.items():
        print(f'Sharpe ratio monthly {key} {np.mean(values - (1+rf)**(1/12)+1)/np.std(values):.4f}')'''

    # Print yearly sharpe ratios
    for key, values in monthly_returns.items():
        print(f'Sharpe ratio yearly {key} {np.mean(values - rf)/np.std(values)*(12)**(1/2):.2f}')

'''    print(f'Average share of long expired {np.mean(long_expired_share):.4f}')
    print(f'Average share of short expired {np.mean(short_expired_share):.4f}')'''

"    print(f'Average share of long expired {np.mean(long_expired_share):.4f}')\n    print(f'Average share of short expired {np.mean(short_expired_share):.4f}')"

In [36]:
model_names = ['LSTM-MLP', 'Heston', 'IV', 'GARCH', 'Rolling']
columns = ['Quote_date', 'Price', 'Underlying_last', 'Strike', 'TTM', 'Expiry_date', 'R', 'Option_ID', 'YearMonth', 'Option_exercise', 'Delta', 'SP_expiry', 'Hedge_cost']

for model in model_names:
    df_copy = df_read.copy()
    df_copy = df_copy[columns + [model]]
    df_copy = df_copy.rename(columns={model: 'Prediction'})
    df_copy['Relative_pricing_error'] = (df_copy['Prediction'] - df_copy['Price'])/df_copy['Price']
    print(f'Backtesting model {model}')
    backtest(df_copy)  



Backtesting model LSTM-MLP
Average monthly return long 4.65 %
Average monthly return short -1.50 %
Average monthly return combined 1.58 %
Sharpe ratio yearly long 0.29
Sharpe ratio yearly short -0.47
Sharpe ratio yearly combined -0.12
Backtesting model Heston
Average monthly return long 4.22 %
Average monthly return short -0.75 %
Average monthly return combined 1.74 %
Sharpe ratio yearly long 0.19
Sharpe ratio yearly short -0.62
Sharpe ratio yearly combined -0.05
Backtesting model IV
Average monthly return long 2.76 %
Average monthly return short -1.55 %
Average monthly return combined 0.61 %
Sharpe ratio yearly long 0.08
Sharpe ratio yearly short -0.66
Sharpe ratio yearly combined -0.31
Backtesting model GARCH
Average monthly return long 3.55 %
Average monthly return short -1.06 %
Average monthly return combined 1.24 %
Sharpe ratio yearly long 0.13
Sharpe ratio yearly short -0.90
Sharpe ratio yearly combined -0.14
Backtesting model Rolling
Average monthly return long 1.32 %
Average mo

In [12]:
df = df_read.copy()

monthly_returns_long = []
monthly_returns_short = []
monthly_returns_long_hedge = []
monthly_returns_short_hedge = []
monthly_returns_long_portfolio = []
monthly_returns_short_portfolio = []

rf = 0.02

# Group by month
grouped = df.groupby('YearMonth')

for name, group in grouped:
    first_day = group['Quote_date'].iloc[0]
    df_start = group[group['Quote_date'] == first_day]
    df_start = df_start[df_start['Price'] >= 5]
    df_start = df_start[df_start['TTM'] >= 5/365]

    last_trading_day = group['Quote_date'].iloc[-1]
    #print(f'Last trading day: {last_trading_day}')
    #last_trading_day = group['Quote_date'].unique()[10]
    df_final = group[group['Quote_date'] == last_trading_day][['Price', 'Option_ID']]
    df_final.rename(columns={'Price': 'End_price'}, inplace=True)    
    df_start = df_start.merge(df_final, how='left', on='Option_ID')
    df_start = df_start[~((df_start['Expiry_date'] > last_trading_day) & (df_start['End_price'].isnull()))]

    # Select the top 10% underpriced and overpriced options
    underpriced = df_start.nlargest(int(0.05*len(df_start)), 'Relative_pricing_error')
    overpriced = df_start.nsmallest(int(0.05*len(df_start)), 'Relative_pricing_error')

    '''underpriced = df_start[df_start['Absolute_pricing_error'] >= 1]
    overpriced = df_start[df_start['Absolute_pricing_error'] <= -1]'''
    
    underpriced_expired = underpriced[underpriced['Expiry_date'] <= last_trading_day]
    overpriced_expired = overpriced[overpriced['Expiry_date'] <= last_trading_day]

    underpriced_alive = underpriced[underpriced['Expiry_date'] > last_trading_day]
    overpriced_alive = overpriced[overpriced['Expiry_date'] > last_trading_day]

    #Calculate abs returns
    underpriced_expired['Option_return_abs'] = underpriced_expired['Option_exercise'] - underpriced_expired['Price']
    overpriced_expired['Option_return_abs'] = -(overpriced_expired['Option_exercise'] - overpriced_expired['Price'])

    underpriced_alive['Option_return_abs'] = (underpriced_alive['End_price'] - underpriced_alive['Price'])
    overpriced_alive['Option_return_abs'] = - (overpriced_alive['End_price'] - overpriced_alive['Price'])

    underpriced_expired['Hedge_return_abs'] = -underpriced_expired['Delta'] * (underpriced_expired['SP_expiry'] - underpriced_expired['Underlying_last'])
    overpriced_expired['Hedge_return_abs'] = overpriced_expired['Delta'] * (overpriced_expired['SP_expiry'] - overpriced_expired['Underlying_last'])

    underpriced_alive['Hedge_return_abs'] = - underpriced_alive['Delta'] * (underpriced_alive['Underlying_last'] - underpriced_alive['End_price'])
    overpriced_alive['Hedge_return_abs'] = overpriced_alive['Delta'] * (overpriced_alive['Underlying_last'] - overpriced_alive['End_price'])

    # Calculate pct returns
    underpriced_expired['Option_return_pct'] = underpriced_expired['Option_return_abs'] / underpriced_expired['Price']
    overpriced_expired['Option_return_pct'] = overpriced_expired['Option_return_abs'] / overpriced_expired['Price']

    underpriced_alive['Option_return_pct'] = underpriced_alive['Option_return_abs'] / underpriced_alive['Price']
    overpriced_alive['Option_return_pct'] = overpriced_alive['Option_return_abs'] / overpriced_alive['Price']

    underpriced_expired['Hedge_return_pct'] = underpriced_expired['Hedge_return_abs'] / underpriced_expired['Hedge_cost']
    overpriced_expired['Hedge_return_pct'] = overpriced_expired['Hedge_return_abs'] / overpriced_expired['Hedge_cost']

    underpriced_alive['Hedge_return_pct'] = underpriced_alive['Hedge_return_abs'] / underpriced_alive['Hedge_cost']
    overpriced_alive['Hedge_return_pct'] = overpriced_alive['Hedge_return_abs'] / overpriced_alive['Hedge_cost']

    underpriced_expired['Portfolio_return_pct'] = (underpriced_expired['Option_return_abs'] + underpriced_expired['Hedge_return_abs']) / (underpriced_expired['Price'] + underpriced_expired['Hedge_cost'])
    overpriced_expired['Portfolio_return_pct'] = (overpriced_expired['Option_return_abs'] + overpriced_expired['Hedge_return_abs']) / (overpriced_expired['Price'] + overpriced_expired['Hedge_cost'])

    underpriced_alive['Portfolio_return_pct'] = (underpriced_alive['Option_return_abs'] + underpriced_alive['Hedge_return_abs']) / (underpriced_alive['Price'] + underpriced_expired['Hedge_cost'])
    overpriced_alive['Portfolio_return_pct'] = (overpriced_alive['Option_return_abs'] + overpriced_alive['Hedge_return_abs']) / (overpriced_alive['Price'] + overpriced_expired['Hedge_cost'])

    # Calculate average return
    underpriced_return = 0
    overpriced_return = 0
    underpriced_hedge_return = 0
    overpriced_hedge_return = 0
    underpriced_portfolio_return = 0
    overpriced_portfolio_return = 0

    if len(underpriced_expired) > 0:
        underpriced_return += underpriced_expired['Option_return_pct'].sum()
        overpriced_hedge_return += underpriced_expired['Hedge_return_pct'].sum()
        underpriced_portfolio_return += underpriced_expired['Portfolio_return_pct'].sum()
    if len(underpriced_alive) > 0:
        underpriced_return += underpriced_alive['Option_return_pct'].sum()
        overpriced_hedge_return += underpriced_alive['Hedge_return_pct'].sum()
        underpriced_portfolio_return += underpriced_alive['Portfolio_return_pct'].sum()   
    if len(overpriced_expired) > 0:
        overpriced_return += overpriced_expired['Option_return_pct'].sum()
        underpriced_hedge_return += overpriced_expired['Hedge_return_pct'].sum()
        overpriced_portfolio_return += overpriced_expired['Portfolio_return_pct'].sum()
    if len(overpriced_alive) > 0:
        overpriced_return += overpriced_alive['Option_return_pct'].sum()
        underpriced_hedge_return += overpriced_alive['Hedge_return_pct'].sum()
        overpriced_portfolio_return += overpriced_alive['Portfolio_return_pct'].sum()

    underpriced_return = underpriced_return / len(underpriced) if len(underpriced) > 0 else 0
    overpriced_return = overpriced_return / len(overpriced) if len(overpriced) > 0 else 0
    underpriced_hedge_return = underpriced_hedge_return / len(underpriced) if len(underpriced) > 0 else 0
    overpriced_hedge_return = overpriced_hedge_return / len(overpriced) if len(overpriced) > 0 else 0
    underpriced_portfolio_return = underpriced_portfolio_return / len(underpriced) if len(underpriced) > 0 else 0
    overpriced_portfolio_return = overpriced_portfolio_return / len(overpriced) if len(overpriced) > 0 else 0


    # Store returns
    monthly_returns_long.append(underpriced_return)
    monthly_returns_short.append(overpriced_return)
    monthly_returns_long_hedge.append(underpriced_hedge_return)
    monthly_returns_short_hedge.append(overpriced_hedge_return)
    monthly_returns_long_portfolio.append(underpriced_portfolio_return)
    monthly_returns_short_portfolio.append(overpriced_portfolio_return)

print(monthly_returns_long)
print(monthly_returns_short)
print(monthly_returns_long_hedge)
print(monthly_returns_short_hedge)
print(monthly_returns_long_portfolio)
print(monthly_returns_short_portfolio)

# Convert lists to arrays for easier calculations
monthly_returns_long = np.array(monthly_returns_long)
monthly_returns_short = np.array(monthly_returns_short)
monthly_returns_long_hedge = np.array(monthly_returns_long_hedge)
monthly_returns_short_hedge = np.array(monthly_returns_short_hedge)
monthly_returns_long_portfolio = np.array(monthly_returns_long_portfolio)
monthly_returns_short_portfolio = np.array(monthly_returns_short_portfolio)

monthly_returns_combined = 0.5*monthly_returns_long + 0.5*monthly_returns_short
monthly_returns_combined_hedge = 0.5*monthly_returns_long_hedge + 0.5*monthly_returns_short_hedge
monthly_returns_combined_hedged_portfolio = 0.5*monthly_returns_long_portfolio + 0.5*monthly_returns_short_portfolio

# Calculate average monthly return and annualized return
average_monthly_return_long = np.mean(monthly_returns_long)
average_monthly_return_short = np.mean(monthly_returns_short)
average_monthly_return_combined = np.mean(monthly_returns_combined)
average_monthly_return_long_hedge = np.mean(monthly_returns_long_hedge)
average_monthly_return_short_hedge = np.mean(monthly_returns_short_hedge)
average_monthly_return_combined_hedge = np.mean(monthly_returns_combined_hedge)
average_monthly_return_long_portfolio = np.mean(monthly_returns_long_portfolio)
average_monthly_return_short_portfolio = np.mean(monthly_returns_short_portfolio)
average_monthly_return_combined_hedged_portfolio = np.mean(monthly_returns_combined_hedged_portfolio)

annualized_return_long = (1 + average_monthly_return_long) ** 12 - 1
annualized_return_short = (1 + average_monthly_return_short) ** 12 - 1
annualized_return_combined = (1 + average_monthly_return_combined) ** 12 - 1
annualized_return_long_hedge = (1 + average_monthly_return_long_hedge) ** 12 - 1
annualized_return_short_hedge = (1 + average_monthly_return_short_hedge) ** 12 - 1
annualized_return_combined_hedge = (1 + average_monthly_return_combined_hedge) ** 12 - 1
annualized_return_long_hedged_portfolio = (1 + average_monthly_return_long_portfolio) ** 12 - 1
annualized_return_short_hedged_portfolio = (1 + average_monthly_return_short_portfolio) ** 12 - 1
annualized_return_combined_hedged_portfolio = (1 + average_monthly_return_combined_hedged_portfolio) ** 12 - 1

print('Average monthly return (long): ', average_monthly_return_long)
print('Average monthly return (short): ', average_monthly_return_short)
print('Average monthly return (combined): ', average_monthly_return_combined)
print('Average monthly return (long hedge): ', average_monthly_return_long_hedge)
print('Average monthly return (short hedge): ', average_monthly_return_short_hedge)
print('Average monthly return (combined hedge): ', average_monthly_return_combined_hedge)
print('Average monthly return (long hedged portfolio): ', average_monthly_return_long_portfolio)
print('Average monthly return (short hedged portfolio): ', average_monthly_return_short_portfolio)
print('Average monthly return (combined hedged portfolio): ', average_monthly_return_combined_hedged_portfolio)

print('Annualized return (long): ', annualized_return_long)
print('Annualized return (short): ', annualized_return_short)
print('Annualized return (combined): ', annualized_return_combined)
print('Annualized return (long hedge): ', annualized_return_long_hedge)
print('Annualized return (short hedge): ', annualized_return_short_hedge)
print('Annualized return (combined hedge): ', annualized_return_combined_hedge)
print('Annualized return (long hedged portfolio): ', annualized_return_long_hedged_portfolio)
print('Annualized return (short hedged portfolio): ', annualized_return_short_hedged_portfolio)
print('Annualized return (combined hedged portfolio): ', annualized_return_combined_hedged_portfolio)

print('Monthly sharpe ratio (long): ', (average_monthly_return_long - (1+rf)**(1/12) + 1) / np.std(monthly_returns_long))
print('Monthly sharpe ratio (short): ', (average_monthly_return_short - (1+rf)**(1/12) + 1) / np.std(monthly_returns_short))
print('Monthly sharpe ratio (combined): ', (average_monthly_return_combined - (1+rf)**(1/12) + 1) / np.std(monthly_returns_combined))
print('Monthly sharpe ratio (long hedge): ', (average_monthly_return_long_hedge - (1+rf)**(1/12) + 1) / np.std(monthly_returns_long_hedge))
print('Monthly sharpe ratio (short hedge): ', (average_monthly_return_short_hedge - (1+rf)**(1/12) + 1) / np.std(monthly_returns_short_hedge))
print('Monthly sharpe ratio (combined hedge): ', (average_monthly_return_combined_hedge - (1+rf)**(1/12) + 1) / np.std(monthly_returns_combined_hedge))
print('Monthly sharpe ratio (long hedged portfolio): ', (average_monthly_return_long_portfolio - (1+rf)**(1/12) + 1) / np.std(monthly_returns_long_portfolio))
print('Monthly sharpe ratio (short hedged portfolio): ', (average_monthly_return_short_portfolio - (1+rf)**(1/12) + 1) / np.std(monthly_returns_short_portfolio))
print('Monthly sharpe ratio (combined hedged portfolio): ', (average_monthly_return_combined_hedged_portfolio - (1+rf)**(1/12) + 1) / np.std(monthly_returns_combined_hedged_portfolio))

print('Annual Sharpe ratio (long): ', (annualized_return_long -rf) / np.std(monthly_returns_long)*(12**0.5))
print('Annual Sharpe ratio (short): ', (annualized_return_short-rf) / np.std(monthly_returns_short)*(12**0.5))
print('Annual Sharpe ratio (combined): ', (annualized_return_combined-rf) / np.std(monthly_returns_combined)*(12**0.5))
print('Annual Sharpe ratio (long hedge): ', (annualized_return_long_hedge-rf) / np.std(monthly_returns_long_hedge)*(12**0.5))
print('Annual Sharpe ratio (short hedge): ', (annualized_return_short_hedge-rf) / np.std(monthly_returns_short_hedge)*(12**0.5))
print('Annual Sharpe ratio (combined hedge): ', (annualized_return_combined_hedge-rf) / np.std(monthly_returns_combined_hedge)*(12**0.5))
print('Annual Sharpe ratio (long hedged portfolio): ', (annualized_return_long_hedged_portfolio-rf) / np.std(monthly_returns_long_portfolio)*(12**0.5))
print('Annual Sharpe ratio (short hedged portfolio): ', (annualized_return_short_hedged_portfolio-rf) / np.std(monthly_returns_short_portfolio)*(12**0.5))
print('Annual Sharpe ratio (combined hedged portfolio): ', (annualized_return_combined_hedged_portfolio-rf) / np.std(monthly_returns_combined_hedged_portfolio)*(12**0.5))

[-0.8495224055803183, 1.3498077160665993, -0.6650931691964124, -0.1861370544341343, -0.5819563869889879, -0.5514000213811064, -0.005081190861449057, -0.8380538316463667, 0.1260343686490098, 1.4681879484504574, -0.31153536691095, -0.28486994840974017, -0.6838469483910338, -0.24854856152953084, 0.31000367249720145, -0.28883979478685956, -0.5851007367401645, -0.4195139485911624, 1.7448771499105225, -0.5450325482965298, -0.7346172340135853, -0.7343637176044233, 0.4769315639800677, 0.5403052314224075, 0.165516474793134, 4.242066231188795, -0.14970535447403974, -0.43995558247048566, -0.384254676646186, -0.7519440714013202, 0.4351523137313771, -0.48448186871937576, 0.4511013689519997, 0.453306114459487, 0.6954557313421074, 0.04714823661583379, 8.744295990522726, -0.9993403958709585, 0.5848723041590894, 0.38004705340819783, 0.004569248471512227, -0.4757470862688475, 1.0268489011271524, 2.2042770298812018, -0.4645373252696344, -0.9622624093125576, -0.08372858604438226, -0.8945503413883258, 0.96

In [12]:
df = df_read.copy()

monthly_returns_long = []
monthly_returns_short = []
monthly_returns_combined = []
monthly_returns_long_hedged = []
monthly_returns_short_hedged = []
monthly_returns_combined_hedged = []

# Group by month
grouped = df.groupby('YearMonth')

for name, group in grouped:
    # ... Same code as before for selecting underpriced and overpriced options ...

    # For each option, calculate the number of shares needed to hedge
    underpriced['Hedge'] = underpriced['Delta'] * underpriced['Underlying_last']
    overpriced['Hedge'] = -overpriced['Delta'] * overpriced['Underlying_last']

    # Calculate the returns from the hedge
    underpriced['Hedge_Returns'] = (underpriced['Hedge'] / underpriced['Underlying_last'] - underpriced['Hedge'] / group['Underlying_last'].iloc[-1])
    overpriced['Hedge_Returns'] = (overpriced['Hedge'] / overpriced['Underlying_last'] - overpriced['Hedge'] / group['Underlying_last'].iloc[-1])

    # Calculate total returns including hedge
    underpriced_return_hedged = underpriced_return + underpriced['Hedge_Returns'].sum() / len(underpriced)
    overpriced_return_hedged = overpriced_return + overpriced['Hedge_Returns'].sum() / len(overpriced)

    # Store returns
    monthly_returns_long_hedged.append(underpriced_return_hedged)
    monthly_returns_short_hedged.append(overpriced_return_hedged)
    monthly_returns_combined_hedged.append(0.5*underpriced_return_hedged + 0.5*overpriced_return_hedged)

# Convert lists to arrays for easier calculations
monthly_returns_long_hedged = np.array(monthly_returns_long_hedged)
monthly_returns_short_hedged = np.array(monthly_returns_short_hedged)
monthly_returns_combined_hedged = np.array(monthly_returns_combined_hedged)

# Calculate average monthly return and annualized return
average_monthly_return_long_hedged = np.mean(monthly_returns_long_hedged)
average_monthly_return_short_hedged = np.mean(monthly_returns_short_hedged)
average_monthly_return_combined_hedged = np.mean(monthly_returns_combined_hedged)

annualized_return_long_hedged = (1 + average_monthly_return_long_hedged) ** 12 - 1
annualized_return_short_hedged = (1 + average_monthly_return_short_hedged) ** 12 - 1
annualized_return_combined_hedged = (1 + average_monthly_return_combined_hedged) ** 12 - 1

print('Average monthly return (long, hedged): ', average_monthly_return_long_hedged)
print('Average monthly return (short, hedged): ', average_monthly_return_short_hedged)
print('Average monthly return (combined, hedged): ', average_monthly_return_combined_hedged)

print('Annualized return (long, hedged): ', annualized_return_long_hedged)
print('Annualized return (short, hedged): ', annualized_return_short_hedged)
print('Annualized return (combined, hedged): ', annualized_return_combined_hedged)

print('Monthly sharpe ratio (long, hedged): ', (average_monthly_return_long_hedged - (1+rf)**(1/12) + 1) / np.std(monthly_returns_long_hedged))
print('Monthly sharpe ratio (short, hedged): ', (average_monthly_return_short_hedged - (1+rf)**(1/12) + 1) / np.std(monthly_returns_short_hedged))
print('Monthly sharpe ratio (combined, hedged): ', (average_monthly_return_combined_hedged - (1+rf)**(1/12) + 1) / np.std(monthly_returns_combined_hedged))

print('Annual Sharpe ratio (long, hedged): ', (average_monthly_return_long_hedged -rf) / np.std(monthly_returns_long_hedged)*(12**0.5))
print('Annual Sharpe ratio (short, hedged): ', (average_monthly_return_short_hedged-rf) / np.std(monthly_returns_short_hedged)*(12**0.5))
print('Annual Sharpe ratio (combined, hedged): ', (average_monthly_return_combined_hedged-rf) / np.std(monthly_returns_combined_hedged)*(12**0.5))

NameError: name 'underpriced' is not defined

In [ ]:
df = df_read.copy()

monthly_returns_long = []
monthly_returns_short = []
monthly_returns_long_hedge = []
monthly_returns_short_hedge = []
monthly_roc_long = []
monthly_roc_short = []

rf = 0.02

# Group by month
grouped = df.groupby('YearMonth')

for name, group in grouped:
    first_day = group['Quote_date'].iloc[0]
    df_start = group[group['Quote_date'] == first_day]
    df_start = df_start[df_start['Price'] >= 5]
    df_start = df_start[df_start['TTM'] >= 5/365]

    last_trading_day = group['Quote_date'].iloc[-1]
    #print(f'Last trading day: {last_trading_day}')
    #last_trading_day = group['Quote_date'].unique()[10]
    df_final = group[group['Quote_date'] == last_trading_day][['Price', 'Option_ID']]
    df_final.rename(columns={'Price': 'End_price'}, inplace=True)    
    df_start = df_start.merge(df_final, how='left', on='Option_ID')
    df_start = df_start[~((df_start['Expiry_date'] > last_trading_day) & (df_start['End_price'].isnull()))]

    # Select the top 10% underpriced and overpriced options
    underpriced = df_start.nlargest(int(0.05*len(df_start)), 'Relative_pricing_error')
    overpriced = df_start.nsmallest(int(0.05*len(df_start)), 'Relative_pricing_error')

    '''underpriced = df_start[df_start['Absolute_pricing_error'] >= 1]
    overpriced = df_start[df_start['Absolute_pricing_error'] <= -1]'''

    # Extract s&p500 price at the last trading day
    sp_price = group['Underlying_last'].iloc[-1]

    # For options expiried within the month, calculate return
    
    underpriced_expired = underpriced[underpriced['Expiry_date'] <= last_trading_day]
    overpriced_expired = overpriced[overpriced['Expiry_date'] <= last_trading_day]

    underpriced_alive = underpriced[underpriced['Expiry_date'] > last_trading_day]
    overpriced_alive = overpriced[overpriced['Expiry_date'] > last_trading_day]

    #Calculate returns
    underpriced_expired['Returns'] = (underpriced_expired['SP_expiry'] - underpriced_expired['Strike']).clip(lower=0) / underpriced_expired['Price']
    overpriced_expired['Returns'] = - ((overpriced_expired['SP_expiry'] - overpriced_expired['Strike']).clip(lower=0) / overpriced_expired['Price'])

    underpriced_alive['Returns'] = (underpriced_alive['End_price'] - underpriced_alive['Price']) / underpriced_alive['Price']
    overpriced_alive['Returns'] = (overpriced_alive['Price'] - overpriced_alive['End_price']) / overpriced_alive['Price']

    # Calculate average return
    underpriced_return = 0
    if len(underpriced_expired) > 0:
        underpriced_return += underpriced_expired['Returns'].sum()
    if len(underpriced_alive) > 0:
        underpriced_return += underpriced_alive['Returns'].sum()
    underpriced_return = underpriced_return / len(underpriced) if len(underpriced) > 0 else 0

    overpriced_return = 0
    if len(overpriced_expired) > 0:
        overpriced_return += overpriced_expired['Returns'].sum()
    if len(overpriced_alive) > 0:
        overpriced_return += overpriced_alive['Returns'].sum()
    overpriced_return = overpriced_return / len(overpriced) if len(overpriced) > 0 else 0

    # Store returns
    monthly_returns_long.append(underpriced_return)
    monthly_returns_short.append(overpriced_return)

    # Calculate the returns from the hedge
    underpriced['Hedge_Returns'] = underpriced['Delta']*group['Underlying_last'].iloc[-1] - underpriced['Delta']*underpriced['Underlying_last']
    
    (underpriced['Hedge'] / underpriced['Underlying_last'] - underpriced['Hedge'] / group['Underlying_last'].iloc[-1]) * underpriced['Underlying_last']
    overpriced['Hedge_Returns'] = (overpriced['Hedge'] / overpriced['Underlying_last'] - overpriced['Hedge'] / group['Underlying_last'].iloc[-1]) * overpriced['Underlying_last']

    # Calculate the total return for each option and its hedge
    underpriced['Total_Returns'] = underpriced_alive['Returns'] * underpriced_expired['Price'] + underpriced_expired['Returns']*underpriced_expired['Price'] + underpriced['Hedge_Returns']
    
    overpriced['Total_Returns'] = overpriced['Returns'] * overpriced['Price'] + overpriced['Hedge_Returns']

    # Calculate the return on capital for each option and its hedge
    underpriced['ROC'] = underpriced['Total_Returns'] / underpriced['Total_cost']
    overpriced['ROC'] = overpriced['Total_Returns'] / overpriced['Total_cost']

    # Calculate the average return on capital
    underpriced_roc = underpriced['ROC'].mean()
    overpriced_roc = overpriced['ROC'].mean()

    # Store returns
    monthly_roc_long.append(underpriced_roc)
    monthly_roc_short.append(overpriced_roc)



print(monthly_returns_long)
print(monthly_returns_short)


# Convert lists to arrays for easier calculations
monthly_returns_long = np.array(monthly_returns_long)
monthly_returns_short = np.array(monthly_returns_short)
monthly_returns_long_hedge = np.array(monthly_returns_long_hedge)
monthly_returns_short_hedge = np.array(monthly_returns_short_hedge)
monthly_roc_long = np.array(monthly_roc_long)
monthly_roc_short = np.array(monthly_roc_short)
print(monthly_roc_long.mean())
print(monthly_roc_short.mean())
print(monthly_roc_long.mean()*0.5 + monthly_roc_long.mean()*0.5)

monthly_returns_combined = 0.5*monthly_returns_long + 0.5*monthly_returns_short
monthly_returns_combined_hedge = 0.5*monthly_returns_long_hedge + 0.5*monthly_returns_short_hedge

monthly_returns_long_hedged_portfolio = monthly_returns_long + monthly_returns_long_hedge
monthly_returns_short_hedged_portfolio = monthly_returns_short + monthly_returns_short_hedge
monthly_returns_combined_hedged_portfolio = monthly_returns_combined + monthly_returns_combined_hedge

# Calculate average monthly return and annualized return
average_monthly_return_long = np.mean(monthly_returns_long)
average_monthly_return_short = np.mean(monthly_returns_short)
average_monthly_return_combined = np.mean(monthly_returns_combined)
average_monthly_return_long_hedge = np.mean(monthly_returns_long_hedge)
average_monthly_return_short_hedge = np.mean(monthly_returns_short_hedge)
average_monthly_return_combined_hedge = np.mean(monthly_returns_combined_hedge)
average_monthly_return_long_hedged_portfolio = np.mean(monthly_returns_long_hedged_portfolio)
average_monthly_return_short_hedged_portfolio = np.mean(monthly_returns_short_hedged_portfolio)
average_monthly_return_combined_hedged_portfolio = np.mean(monthly_returns_combined_hedged_portfolio)

annualized_return_long = (1 + average_monthly_return_long) ** 12 - 1
annualized_return_short = (1 + average_monthly_return_short) ** 12 - 1
annualized_return_combined = (1 + average_monthly_return_combined) ** 12 - 1
annualized_return_long_hedge = (1 + average_monthly_return_long_hedge) ** 12 - 1
annualized_return_short_hedge = (1 + average_monthly_return_short_hedge) ** 12 - 1
annualized_return_combined_hedge = (1 + average_monthly_return_combined_hedge) ** 12 - 1
annualized_return_long_hedged_portfolio = (1 + average_monthly_return_long_hedged_portfolio) ** 12 - 1
annualized_return_short_hedged_portfolio = (1 + average_monthly_return_short_hedged_portfolio) ** 12 - 1
annualized_return_combined_hedged_portfolio = (1 + average_monthly_return_combined_hedged_portfolio) ** 12 - 1


print('Average monthly return (long): ', average_monthly_return_long)
print('Average monthly return (short): ', average_monthly_return_short)
print('Average monthly return (combined): ', average_monthly_return_combined)
print('Average monthly return (long hedge): ', average_monthly_return_long_hedge)
print('Average monthly return (short hedge): ', average_monthly_return_short_hedge)
print('Average monthly return (combined hedge): ', average_monthly_return_combined_hedge)
print('Average monthly return (long hedged portfolio): ', average_monthly_return_long_hedged_portfolio)
print('Average monthly return (short hedged portfolio): ', average_monthly_return_short_hedged_portfolio)
print('Average monthly return (combined hedged portfolio): ', average_monthly_return_combined_hedged_portfolio)


print('Annualized return (long): ', annualized_return_long)
print('Annualized return (short): ', annualized_return_short)
print('Annualized return (combined): ', annualized_return_combined)
print('Annualized return (long hedge): ', annualized_return_long_hedge)
print('Annualized return (short hedge): ', annualized_return_short_hedge)
print('Annualized return (combined hedge): ', annualized_return_combined_hedge)
print('Annualized return (long hedged portfolio): ', annualized_return_long_hedged_portfolio)
print('Annualized return (short hedged portfolio): ', annualized_return_short_hedged_portfolio)
print('Annualized return (combined hedged portfolio): ', annualized_return_combined_hedged_portfolio)

print('Monthly sharpe ratio (long): ', (average_monthly_return_long - (1+rf)**(1/12) + 1) / np.std(monthly_returns_long))
print('Monthly sharpe ratio (short): ', (average_monthly_return_short - (1+rf)**(1/12) + 1) / np.std(monthly_returns_short))
print('Monthly sharpe ratio (combined): ', (average_monthly_return_combined - (1+rf)**(1/12) + 1) / np.std(monthly_returns_combined))
print('Monthly sharpe ratio (long hedge): ', (average_monthly_return_long_hedge - (1+rf)**(1/12) + 1) / np.std(monthly_returns_long_hedge))
print('Monthly sharpe ratio (short hedge): ', (average_monthly_return_short_hedge - (1+rf)**(1/12) + 1) / np.std(monthly_returns_short_hedge))
print('Monthly sharpe ratio (combined hedge): ', (average_monthly_return_combined_hedge - (1+rf)**(1/12) + 1) / np.std(monthly_returns_combined_hedge))
print('Monthly sharpe ratio (long hedged portfolio): ', (average_monthly_return_long_hedged_portfolio - (1+rf)**(1/12) + 1) / np.std(monthly_returns_long_hedged_portfolio))
print('Monthly sharpe ratio (short hedged portfolio): ', (average_monthly_return_short_hedged_portfolio - (1+rf)**(1/12) + 1) / np.std(monthly_returns_short_hedged_portfolio))
print('Monthly sharpe ratio (combined hedged portfolio): ', (average_monthly_return_combined_hedged_portfolio - (1+rf)**(1/12) + 1) / np.std(monthly_returns_combined_hedged_portfolio))

print('Annual Sharpe ratio (long): ', (average_monthly_return_long -rf) / np.std(monthly_returns_long)*(12**0.5))
print('Annual Sharpe ratio (short): ', (average_monthly_return_short-rf) / np.std(monthly_returns_short)*(12**0.5))
print('Annual Sharpe ratio (combined): ', (average_monthly_return_combined-rf) / np.std(monthly_returns_combined)*(12**0.5))
print('Annual Sharpe ratio (long hedge): ', (average_monthly_return_long_hedge-rf) / np.std(monthly_returns_long_hedge)*(12**0.5))
print('Annual Sharpe ratio (short hedge): ', (average_monthly_return_short_hedge-rf) / np.std(monthly_returns_short_hedge)*(12**0.5))
print('Annual Sharpe ratio (combined hedge): ', (average_monthly_return_combined_hedge-rf) / np.std(monthly_returns_combined_hedge)*(12**0.5))
print('Annual Sharpe ratio (long hedged portfolio): ', (average_monthly_return_long_hedged_portfolio-rf) / np.std(monthly_returns_long_hedged_portfolio)*(12**0.5))
print('Annual Sharpe ratio (short hedged portfolio): ', (average_monthly_return_short_hedged_portfolio-rf) / np.std(monthly_returns_short_hedged_portfolio)*(12**0.5))
print('Annual Sharpe ratio (combined hedged portfolio): ', (average_monthly_return_combined_hedged_portfolio-rf) / np.std(monthly_returns_combined_hedged_portfolio)*(12**0.5))

In [ ]:
transaction_cost_options = 0.0002
transaction_cost_stock = 0.0002


df = df_read.copy()

# Lists to store returns
monthly_returns = {
    'long': np.array([]),
    'short': np.array([]),
    'long_hedge': np.array([]),
    'short_hedge': np.array([]),
    'long_portfolio': np.array([]),
    'short_portfolio': np.array([])
}

long_expired_share = np.array([])
short_expired_share = np.array([])

rf = 0.02

# Group by month
grouped = df.groupby('YearMonth')

for name, group in grouped:
    first_day = group['Quote_date'].iloc[0]
    df_start = group[group['Quote_date'] == first_day]
    #df_start = df_start[df_start['Price'] >= 5]
    #df_start = df_start[df_start['TTM'] >= 5/365]

    last_trading_day = group['Quote_date'].iloc[-1]
    #last_trading_day = group['Quote_date'].unique()[5]
    df_final = group[group['Quote_date'] == last_trading_day][['Price', 'Option_ID']]
    df_final.rename(columns={'Price': 'End_price'}, inplace=True)    
    df_start = df_start.merge(df_final, how='left', on='Option_ID')
    df_start = df_start[~((df_start['Expiry_date'] > last_trading_day) & (df_start['End_price'].isnull()))]

    # Select the top 10% underpriced and overpriced options
    underpriced = df_start.nlargest(int(0.1*len(df_start)), 'Relative_pricing_error')
    overpriced = df_start.nsmallest(int(0.1*len(df_start)), 'Relative_pricing_error')

    '''underpriced = df_start[df_start['Absolute_pricing_error'] >= 1]
    overpriced = df_start[df_start['Absolute_pricing_error'] <= -1]'''
    
    underpriced_expired = underpriced[underpriced['Expiry_date'] <= last_trading_day]
    overpriced_expired = overpriced[overpriced['Expiry_date'] <= last_trading_day]

    underpriced_alive = underpriced[underpriced['Expiry_date'] > last_trading_day]
    overpriced_alive = overpriced[overpriced['Expiry_date'] > last_trading_day]

    long_expired_share = np.append(long_expired_share, len(underpriced_expired)/len(underpriced))
    short_expired_share = np.append(short_expired_share, len(overpriced_expired)/len(overpriced))

    #Calculate abs returns
    underpriced_expired.loc[:, 'Option_return_abs'] = underpriced_expired['Option_exercise'] - underpriced_expired['Price'] - transaction_cost_options*underpriced_expired['Price']
    overpriced_expired.loc[:, 'Option_return_abs'] = -(overpriced_expired['Option_exercise'] - overpriced_expired['Price']) - transaction_cost_options*underpriced_expired['Price']

    underpriced_alive.loc[:, 'Option_return_abs'] = (underpriced_alive['End_price'] - underpriced_alive['Price']) - transaction_cost_options*(underpriced_alive['Price'] + underpriced_alive['End_price'])
    overpriced_alive.loc[:, 'Option_return_abs'] = - (overpriced_alive['End_price'] - overpriced_alive['Price']) - transaction_cost_options*(overpriced_alive['Price'] + overpriced_alive['End_price'])

    underpriced_expired.loc[:, 'Hedge_return_abs'] = -underpriced_expired['Delta'] * (underpriced_expired['SP_expiry'] - underpriced_expired['Underlying_last']) - transaction_cost_stock*(underpriced_expired['SP_expiry'] + underpriced_expired['Underlying_last'])
    overpriced_expired.loc[:, 'Hedge_return_abs'] = overpriced_expired['Delta'] * (overpriced_expired['SP_expiry'] - overpriced_expired['Underlying_last']) - transaction_cost_stock*(underpriced_expired['SP_expiry'] + underpriced_expired['Underlying_last'])

    sp500_final = group[group['Quote_date'] == last_trading_day]['Underlying_last'].iloc[-1]
    underpriced_alive.loc[:, 'Hedge_return_abs'] = - underpriced_alive['Delta'] * (sp500_final - underpriced_alive['Underlying_last']) - transaction_cost_stock*(sp500_final + underpriced_alive['Underlying_last'])
    overpriced_alive.loc[:, 'Hedge_return_abs'] = overpriced_alive['Delta'] * (sp500_final - overpriced_alive['Underlying_last']) - transaction_cost_stock*(sp500_final + overpriced_alive['Underlying_last'])

    # Calculate pct returns
    underpriced_expired.loc[:, 'Option_return_pct'] = underpriced_expired['Option_return_abs'] / underpriced_expired['Price']
    overpriced_expired.loc[:, 'Option_return_pct'] = overpriced_expired['Option_return_abs'] / overpriced_expired['Price']

    underpriced_alive.loc[:, 'Option_return_pct'] = underpriced_alive['Option_return_abs'] / underpriced_alive['Price']
    overpriced_alive.loc[:, 'Option_return_pct'] = overpriced_alive['Option_return_abs'] / overpriced_alive['Price']

    underpriced_expired.loc[:, 'Hedge_return_pct'] = underpriced_expired['Hedge_return_abs'] / (underpriced_expired['Delta']*underpriced_expired['Underlying_last'])
    overpriced_expired.loc[:, 'Hedge_return_pct'] = overpriced_expired['Hedge_return_abs'] / (overpriced_expired['Delta']*overpriced_expired['Underlying_last'])

    underpriced_alive.loc[:, 'Hedge_return_pct'] = underpriced_alive['Hedge_return_abs'] / (underpriced_alive['Delta']*underpriced_alive['Underlying_last'])
    overpriced_alive.loc[:, 'Hedge_return_pct'] = overpriced_alive['Hedge_return_abs'] / (overpriced_alive['Delta']*overpriced_alive['Underlying_last'])

    underpriced_expired.loc[:, 'Portfolio_return_pct'] = (underpriced_expired['Option_return_abs'] + underpriced_expired['Hedge_return_abs']) / (underpriced_expired['Price'] + underpriced_expired['Hedge_cost'])
    overpriced_expired.loc[:, 'Portfolio_return_pct'] = (overpriced_expired['Option_return_abs'] + overpriced_expired['Hedge_return_abs']) / (overpriced_expired['Price'] + overpriced_expired['Hedge_cost'])

    underpriced_alive.loc[:, 'Portfolio_return_pct'] = (underpriced_alive['Option_return_abs'] + underpriced_alive['Hedge_return_abs']) / (underpriced_alive['Price'] + underpriced_expired['Hedge_cost'])
    overpriced_alive.loc[:, 'Portfolio_return_pct'] = (overpriced_alive['Option_return_abs'] + overpriced_alive['Hedge_return_abs']) / (overpriced_alive['Price'] + overpriced_expired['Hedge_cost'])

    monthly_returns['long'] = np.append(monthly_returns['long'], (underpriced_expired['Option_return_pct'].sum() + underpriced_alive['Option_return_pct'].sum()) / len(underpriced))
    monthly_returns['short'] = np.append(monthly_returns['short'], (overpriced_expired['Option_return_pct'].sum() + overpriced_alive['Option_return_pct'].sum()) / len(overpriced))
    monthly_returns['long_hedge'] = np.append(monthly_returns['long_hedge'], (underpriced_expired['Hedge_return_pct'].sum() + underpriced_alive['Hedge_return_pct'].sum()) / len(underpriced))
    monthly_returns['short_hedge'] = np.append(monthly_returns['short_hedge'], (overpriced_expired['Hedge_return_pct'].sum() + overpriced_alive['Hedge_return_pct'].sum()) / len(overpriced))
    monthly_returns['long_portfolio'] = np.append(monthly_returns['long_portfolio'], (underpriced_expired['Portfolio_return_pct'].sum() + underpriced_alive['Portfolio_return_pct'].sum()) / len(underpriced))
    monthly_returns['short_portfolio'] = np.append(monthly_returns['short_portfolio'], (overpriced_expired['Portfolio_return_pct'].sum() + overpriced_alive['Portfolio_return_pct'].sum()) / len(overpriced))
    
# Add combined returns
monthly_returns['Combined'] = 0.5*monthly_returns['long'] + 0.5*monthly_returns['short']
monthly_returns['Combined_hedge'] = 0.5*monthly_returns['long_hedge'] + 0.5*monthly_returns['short_hedge']
monthly_returns['Combined_hedged_portfolio'] = 0.5*monthly_returns['long_portfolio'] + 0.5*monthly_returns['short_portfolio']

# Print values
'''for values in monthly_returns.items():
    print(values)'''

# Print average monthly returns
for key, values in monthly_returns.items():
    print(f'Average monthly return {key} {np.mean(values):.4f}')

# Print annualized returns
for key, values in monthly_returns.items():
    annualized = (1 + values) ** 12 - 1
    print(f'Annualized return {key} {np.mean(annualized):.4f}')

# Print Sharpe ratios
for key, values in monthly_returns.items():
    print(f'Sharpe ratio monthly {key} {np.mean(values)/np.std(values):.4f}')

# Print yearly sharpe ratios
for key, values in monthly_returns.items():
    print(f'Sharpe ratio yearly {key} {np.mean(values)/np.std(values)*(12)**(1/2):.4f}')

print(f'Average share of overpriced expired {np.mean(long_expired_share):.4f}')
print(f'Average share of overpriced alive {np.mean(short_expired_share):.4f}')